In [160]:
import requests
import lxml.html as lh
import pandas as pd

### 1- First, it's necessary to extract all elements of the wikipedia table.

In [161]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

### and put them into a list

In [162]:
col=[]

i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
   

In [163]:

for j in range(1,len(tr_elements)):
   
    T=tr_elements[j]
    
    if len(T)!=3:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
          
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1
        

### 2- We set the columns and clean the information.

In [164]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.columns = ['Postcode', 'Borough','Neighbourhood']
df = df.replace(r'\s', ' ', regex = True)


In [165]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Which includes to take away the not assigned data in Borough

In [166]:
df = df.drop(df[df['Borough']=='Not assigned'].index)
df = df.reset_index(drop=True)

In [167]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### 3- We combine rows with same Postcode into one row with the neighborhoods separated with a comma.

In [168]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()



### 4- And then replace not assigned neighbourhood for the Borough 

In [170]:
df['Neighbourhood'].replace(to_replace="Not assigned", value=df['Borough'], inplace=True)

In [185]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park"
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge"
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W..."
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


### 5- Finally, we get the shape.

In [156]:
df.shape

(103, 3)